In [32]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import pandas as pd
import random
import sys
import io
import string, re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import itertools

def remove_punc(lyrics):
    return "".join([c for c in lyrics if c not in string.punctuation])
def remove_stopwords(lyrics):
    return [w for w in lyrics if w not in stopwords.words('english')]

markers = ['[', ']','Verse' ,'1', '2','3', 'Chorus', 'Spoken Intro', 'Intro', 'Bridge', 'Outro']

def remove_markers(lyrics):
    return [w for w in lyrics if w not in markers]


lyrics_df = pd.read_csv('data/lyrics.csv')
tokenizer = RegexpTokenizer(r'\w+')

lyrics_df['lyrics'] = lyrics_df['lyrics'].apply(lambda x: remove_punc(x))
lyrics_df['lyrics'] = lyrics_df['lyrics'].apply(lambda x: tokenizer.tokenize(x))
lyrics_df['lyrics'] = lyrics_df['lyrics'].apply(lambda x: remove_markers(x))
#lyrics_df['lyrics'] = lyrics_df['lyrics'].apply(lambda x: remove_stopwords(x))

text = lyrics_df['lyrics'].values
text = list(itertools.chain.from_iterable(text))
text = ' '.join(text)
print(text)
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 60
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1



Youve never met before But still she greets you like a long lost rock n roll Shes definitely one of those Where youll go wherever she goes And with my body on my mind Both stop talking at the exact same time Trying to think of ways to make her mine But theyre difficult to find Its not what I need No your loves not what I need So dont give it to me And she said Oh well I know this will sound cold but I really have to go No its not that Im not free theres nowhere I need to be Its just your loves not what I need so dont give it to me Its not what I need No your loves not what I need So dont give it to me And she said Baby how can I believe you How can I believe you when you cant believe your luck No point sticking to the plan when its come unstuck Its more a hunger than a thirst Shell break your heart the second time Before you know about the first Looks well equipped to leave you in the lurch But youd let her do her worst And its not hard to tell Its obvious the other angels faces fell W

In [42]:

# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    if(((epoch+1)%15) != 0 ):
        return
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)



Build model...


In [43]:
model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])

Epoch 1/60
53622/53622 [==============================] - 14s 270us/step - loss: 2.2797

Epoch 2/60
53622/53622 [==============================] - 13s 247us/step - loss: 1.9262

Epoch 3/60
53622/53622 [==============================] - 13s 236us/step - loss: 1.6899

Epoch 4/60
53622/53622 [==============================] - 13s 236us/step - loss: 1.5604

Epoch 5/60
53622/53622 [==============================] - 13s 237us/step - loss: 1.4608

Epoch 6/60
53622/53622 [==============================] - 13s 235us/step - loss: 1.3812

Epoch 7/60
53622/53622 [==============================] - 13s 235us/step - loss: 1.3183

Epoch 8/60
53622/53622 [==============================] - 13s 237us/step - loss: 1.2694

Epoch 9/60
53622/53622 [==============================] - 13s 236us/step - loss: 1.2328

Epoch 10/60
53622/53622 [==============================] - 13s 238us/step - loss: 1.2018

Epoch 11/60
53622/53622 [==============================] - 13s 236us/step - loss: 1.1730

Epoch 12/60
53622/5

53622/53622 [==============================] - 13s 235us/step - loss: 0.8542

Epoch 41/60
53622/53622 [==============================] - 13s 244us/step - loss: 0.8395

Epoch 42/60
53622/53622 [==============================] - 13s 238us/step - loss: 0.8320

Epoch 43/60
53622/53622 [==============================] - 13s 234us/step - loss: 0.8270

Epoch 44/60
53622/53622 [==============================] - 13s 238us/step - loss: 0.8237

Epoch 45/60
53622/53622 [==============================] - 12s 233us/step - loss: 0.8184

----- Generating text after Epoch: 44
----- diversity: 0.2
----- Generating with seed: " mix Said Youre mistaken if youre thinking That I havent bee"
 mix Said Youre mistaken if youre thinking That I havent been were girls are see might be the strust the train of a there And they said it changes when the sun goes down Around her strunge In the cind and cracks to me back to me I want yourres me when youre high And there wont be arm again But this vormictly was somethin

/home/ggapp/.local/lib/python3.6/site-packages/ipykernel_launcher.py:14: RuntimeWarning: divide by zero encountered in log
  


our houses it easy
----- diversity: 0.5
----- Generating with seed: " This right here I swear will end too soon My limbs seem to "
 This right here I swear will end too soon My limbs seem to hate youve up all too do the strange by underst the pantere forever horrow both good girl dine the languater mome and chaital PreChorus And I wasnt I wanna how down the seat Yeah of a wanna called Cruck in the innt about though Id the day after strange Wout she wont cant the shadin ty the strangate Try now in your store Bruah The only clont the teasy to bring and chaCt the wroughthing about it so 
----- diversity: 1.0
----- Generating with seed: " This right here I swear will end too soon My limbs seem to "
 This right here I swear will end too soon My limbs seem to hire You thoughts you the sightt The op shes They dont own I know the oplen leen Or throw of it Do by brick Waving and queen steah SaWhthere youre to stand my mind no Gook the ves disto from the tyo you down offeptionssed It wanna gote 

In [46]:
#test other achitecture

model = Sequential()
model.add(LSTM(256, input_shape=(maxlen, len(chars)), return_sequences=True))
#model.add(Dropout(0.2))
model.add(LSTM(256, return_sequences=True))
#model.add(Dropout(0.2))
model.add(LSTM(128))
#model.add(Dropout(0.2))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

In [49]:
model.fit(x, y,
          batch_size=256,
          epochs=60,
          callbacks=[print_callback])

Epoch 1/60
53622/53622 [==============================] - 41s 773us/step - loss: 2.6657

Epoch 2/60
53622/53622 [==============================] - 40s 753us/step - loss: 2.3953

Epoch 3/60
53622/53622 [==============================] - 41s 759us/step - loss: 2.2483

Epoch 4/60
53622/53622 [==============================] - 41s 757us/step - loss: 2.1748

Epoch 5/60
53622/53622 [==============================] - 41s 758us/step - loss: 2.1198

Epoch 6/60
53622/53622 [==============================] - 41s 758us/step - loss: 2.0736

Epoch 7/60
53622/53622 [==============================] - 41s 757us/step - loss: 2.0272

Epoch 8/60
53622/53622 [==============================] - 40s 752us/step - loss: 1.9832

Epoch 9/60
53622/53622 [==============================] - 40s 753us/step - loss: 1.9508

Epoch 10/60
53622/53622 [==============================] - 41s 759us/step - loss: 1.9096

Epoch 11/60
53622/53622 [==============================] - 41s 758us/step - loss: 1.8793

Epoch 12/60
53622/5

53622/53622 [==============================] - 43s 798us/step - loss: 1.5913

Epoch 41/60
53622/53622 [==============================] - 42s 785us/step - loss: 1.57960s - loss: 1

Epoch 42/60
53622/53622 [==============================] - 42s 783us/step - loss: 1.5380

Epoch 43/60
53622/53622 [==============================] - 43s 806us/step - loss: 1.5388

Epoch 44/60
53622/53622 [==============================] - 42s 785us/step - loss: 1.5645

Epoch 45/60
53622/53622 [==============================] - 43s 797us/step - loss: 1.3226

----- Generating text after Epoch: 44
----- diversity: 0.2
----- Generating with seed: "us Yeah regardless of whats gone before I want to see all of"
us Yeah regardless of whats gone before I want to see all of the track And the stars of the back on the strange And the shags And the bark of a snake and stare Im sure the stars of a snake and stare He dont the back on the strange And the sun good on the back And theres a seam We like you had a found the star